###**ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA**
This is a trial of mimicking zeyad's finetuning process on aubmindlab/araelectra-base-discriminator for building an arabic qa system.

Zeyad provided me with two scripts:
1. The dataset translator that converts a csv dataset file into 3 clean json files for train, val and test: https://github.com/SubaieiFatemah/Arabic-MRC/blob/main/Translator/translation2dataset.py

*Note: this is my modified github script. The modification is found in lines 23-31. The modification is simply reading the csv file manually from colab instead of using an extrnal link*

2. The finetuning script: https://github.com/zeyadahmed10/Arabic-MRC/blob/main/AraElectraDecoupling-ASQuADv2.ipynb

*Note: I tried to modify the script as shown below in the colab file. The modification is about removing the "plausible answers" and "CLS training" parts as they are needed in his work to predict if unanswarable questions exist or not. However, I do not need those parts.*

FACED ISSUES:
1. IN DATASET SCRIPT:

A. The dataset is small. Solved: increase dataset

B. There is no clear way of how to exactly write the questions and answers but the start of the answer MUST ALWAYS be the exact same of the ground truth found in the context to be able to get the answer start and end correctly. Solved: make them always matching

C. Yes/No answers cause some complications when it comes to getting the answer start as the "yes" and "no" are NOT found in the conext. Solved: eliminate the yes/no part then get the answer start

D. Can the context be simply the answers gathered? or should it have more side information? Solved: yes it can simply be the answers gathered

E. What comes first? Building the contexts or the questions? I saw how SQUAD v1 and v2 were developed and it was building the context first then extracting possible answers. Solved: context comes first from the CGS guidlines pdf file

F. how big can a context be as in how many answers can it have inside of it? There is truncation and batching so clearly there is a limit. How to know what is the limit? for now I have a single context has answers for 60 questions. Solved: for a single context, the context, question and answer MUST all be 512 tokens AFTER tokenization. Thus, keep the contexts, questions and answers as short as possible


2. IN FINETUNING SCRIPT:

A. I tried to run the finetuning script with the CLS part included on 60 questions and got the acc=100 and exact=0.0. why? is this overfitting? or is the dataset too small? Solved: do finetuning on zeyad's code not araelectra and increase the dataset size

B. I tried to run the script again after removing the CLS code blocks and got excat=0.0. i removed it because my dataset is about guidlines and there will always be answers for every question in my dataset. again why zero? is it because the dataset is small? or maybe i messed up the code by removing some parts of it? Solved: the code is the same even after removing the CLS parts and the solution is to do finetuning on zeyad's code not araelectra and increase the dataset size

C. at worst cases can i leave the CLS parts and consider it as a good practice? yes it can be left as is. 

D. How can I finetune ZEYAD'S work and not ARAELECTRA only? I cannot contact zeyad for a month or so because he is on his military service and has no communications. Solved: the code has the modifications in "Running Zeyad's finetuning script on my own dataset - (WITHOUT CLS PARTS)"

E. Running zeyad's original script on his original dataset consumes almost all my ram. Solved: zeyas used a huge dataset: 200k for training and 9.6k for val and 9.6k for test. my dataset is small and will not consume ram. 

F. why is he using span train and train datasets? what is the difference? as i've seen in the code span train is for the dataset training and train is for cls training? Solved: yes. the span train is the training of the dataset. the other parts are deleted. 

G. why is repeating the training process in this way? why didnt he use a loop? he kept injecting the output parameters from one code line into the other. Solved: he used the free colab version and that didnt allow him to train the huge dataset he has in one shot. he had to do it on multiple steps. however, my dataset is small thus, the extra steps are deleted. 


H. if i want my finetuning process to also include a kuwaiti dialect what should i do? will the script still be able to train the model properly if the whole dataset was in kuwaiti dialect? i mean the modification is only going to be on the dataset level? Solved: No. I need to retrain the whole model for adding new vocab and I must refer to the araelectra original paper to do so


I. what is the difference between the functions get raw preds, post raw preds, get preds, get preds 2? Solved: this is a matter of preprocessing the output of the model from token number to strings and some post processing. I don't need to worry about it and I can build my own training script and pre/post-processing (these notebook wasn't meant to be a reference but only for creating state of art model in arabic qa so don't worry about all the details)


J. why use both the freeze method and the .from_pretrained() together? i mean isn't .from_pretrained() alone is sufficient to load the model with its trained weights? or does this mean we don't want the previous model weights to be modified while the finetuning process? i don't even know if ALL the weights change during the finetuning process? and how to decide until which layer we want to freeze? i believe u did 6 layers freeze? Solved: simply he did a lot of experiment in different notebooks also uploaded and referring to literature he used the recommended hyperparameter and freezing 6 layers worked for him, If he loaded the data using from_pretrained without freezing the finetuning will be on the whole model not only the last 6 layers and the output layer

##**Remaining Questions**
1. Should the dataset include the extra diacritics as in hamzah (أ إ آ) or (ة ه)?

2. Should the QUESTION start be included in the ANSWER start? 
Example:

Q: متى يمكن للاساتذة والاساتذة المشاركين المتقاعدين ان يشرفوا على رسائل واطروحات طلبة الدراسات العليا؟ 


A: *يمكن للاساتذة والاساتذة المشاركين المتقاعدين ان يشرفوا على رسائل واطروحات طلبة الدراسات العليا* شريطة عدم ارتباطهم بعقد عمل مع جامعة اخرى


Should i always include the bold part in the answer? Although the answer is what comes after it? Because most of the times it is wordy and i need to keep the answer as short as possibsle (10-20) tokens

3. Should i have question marks in the questions column? Or it doesn't matter? 

4. what if the answer to the question contains a hyperlink?
example:

Q: ما هو الموقع الالكتروني لكلية الدراسات العليا؟

A: الموقع الالكتروني لكلية الدراسات العليا هو www.somelink.com 

### **Installing dependencies**

In [1]:
!pip install transformers
!pip install preprocess
!pip install arabert
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 16.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186471 sha256=75d4de735191934189d0a896a464bd33e7513de6f64fff3bd560293c57eeef30
  Stored in directory: /root/.c

### **Running Zeyad's dataset script on my own dataset**

In [126]:
from arabert.preprocess import ArabertPreprocessor
import pandas as pd
df = pd.read_csv('cgsqa5x.csv')

print(df)

                           title  \
0     تنظيم كلية الدراسات العليا   
1     تنظيم كلية الدراسات العليا   
2     تنظيم كلية الدراسات العليا   
3     تنظيم كلية الدراسات العليا   
4     تنظيم كلية الدراسات العليا   
...                          ...   
1499                 أسئلة شائعة   
1500                 أسئلة شائعة   
1501                 أسئلة شائعة   
1502                 أسئلة شائعة   
1503                 أسئلة شائعة   

                                                context  \
0     رسالة كلية الدراسات العليا هي العمل المخطط اله...   
1     رسالة كلية الدراسات العليا هي العمل المخطط اله...   
2     رسالة كلية الدراسات العليا هي العمل المخطط اله...   
3     رسالة كلية الدراسات العليا هي العمل المخطط اله...   
4     رسالة كلية الدراسات العليا هي العمل المخطط اله...   
...                                                 ...   
1499  يكون موعد التقديم على كلية الدراسات العليا عاد...   
1500  يكون موعد التقديم على كلية الدراسات العليا عاد...   
1501  يكون موعد التقديم على كلية الد

In [125]:
!rm -r Arabic-MRC
!git clone https://github.com/SubaieiFatemah/Arabic-MRC/

Cloning into 'Arabic-MRC'...
remote: Enumerating objects: 461, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 461 (delta 15), reused 0 (delta 0), pack-reused 424
Receiving objects: 100% (461/461), 47.27 MiB | 24.74 MiB/s, done.
Resolving deltas: 100% (215/215), done.
Updating files: 100% (53/53), done.


In [127]:
!python /content/Arabic-MRC/Translator/translation2dataset.py \
#after running this cell cgsqa5x.csv is splitted into /content/asquadv2-train.json /content/asquadv2-val.json /content/asquadv2-test.json

(1504, 8)
Index(['title', 'context', 'question', 'answer', 'answer_start',
       'is_impossible', 'count', 'ID'],
      dtype='object')
(1504, 8)
(902, 8) (301, 8) (301, 8)


### **Running Zeyad's finetuning script on my own dataset - (WITHOUT CLASSIFICATION)**

In [213]:
import os
import shutil
from collections import Counter
import numpy as np
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, ElectraForQuestionAnswering, DataCollatorWithPadding,BertModel, ElectraForSequenceClassification, ElectraModel
from arabert.preprocess import ArabertPreprocessor
import matplotlib.pyplot as plt
import seaborn as sns
import csv
torch.manual_seed(3407)

In [214]:
def add_end_index(answer, context):
  ## 1 if span match the context 0 otherwise
  text = answer['text']
  start_idx = answer['answer_start']
  end_idx = start_idx + len(text)
  answer['answer_end'] = end_idx
  if text == context[start_idx:end_idx]:
    answer['answer_end'] = end_idx
    return False
  for i in range(1,3):
    if text == context[start_idx-i:end_idx-i]:
      answer['answer_end']= end_idx-1
      answer['answer_start'] = start_idx-1
      return False
  return True

In [215]:
def arabert_preprocess(context,question, answer, arabert_prep):
    answer['text'] = arabert_prep.preprocess(answer['text'])
    context = arabert_prep.preprocess(context)
    question = arabert_prep.preprocess(question)
    res = context.find(answer['text'])
    if res !=-1:
        answer['answer_start'] = res
    return context, question, answer, res

In [216]:
def Read_AAQAD(path,arabert_prep):
  contexts =[]
  answers =[]
  questions =[]
  IDs= []
  plausible = []
  cnt = 0
  with open(path) as f:
    aaqad_dict = json.load(f)
    for article in aaqad_dict['data']:
      for passage in article['paragraphs']:
        context = passage['context']
        for qa in passage['qas']:
          question = qa['question']
          if 'plausible_answers' in qa.keys():# there is two cases if the question have no answer then use plausible answer
            access = 'plausible_answers'
            plausible.append(True)
          else:
            access = 'answers'
            plausible.append(False)
          for answer in qa[access]:
            context,question, answer, res =  arabert_preprocess(context,question, answer, arabert_prep)
            #if res==-1:
            #  cnt+=1
            #  continue
            flag = add_end_index(answer, context) #if false dont add the 
            cnt =cnt + flag
            flag = False
            if not flag:
              contexts.append(context)
              answers.append(answer)
              questions.append(question)
              IDs.append(int(qa['id']))
  return contexts,questions,answers,plausible,IDs

why fix them? can i directly use those ids and del id col in original csv?



In [217]:
def fix_ids(path):
  #IDs need to be fixed for evaluating purposes
    a_file = open(path, "r")
    json_object = json.load(a_file)
    a_file.close()
    idx_cnt = 1
    for article in json_object['data']:
      for passage in article['paragraphs']:
        context = passage['context']
        for qa in passage['qas']:
            qa['id'] = str(idx_cnt)
            idx_cnt = idx_cnt + 1
    a_file = open(path, "w")
    json.dump(json_object, a_file)
    a_file.close()

In [218]:
model_name = "araelectra-base-discriminator"
arabert_prep = ArabertPreprocessor(model_name=model_name)
fix_ids('/content/asquadv2-train.json')
fix_ids('/content/asquadv2-val.json')
fix_ids('/content/asquadv2-test.json')
asquad_train_contexts, asquad_train_questions, asquad_train_answers,asquad_train_plausible, asquad_train_ids = Read_AAQAD('/content/asquadv2-train.json', arabert_prep)
asquad_val_contexts, asquad_val_questions, asquad_val_answers,asquad_val_plausible, asquad_val_ids = Read_AAQAD('/content/asquadv2-val.json', arabert_prep)
asquad_test_contexts, asquad_test_questions, asquad_test_answers,asquad_test_plausible, asquad_test_ids = Read_AAQAD('/content/asquadv2-test.json', arabert_prep)

google Semantic Search — Sentence-Transformers documentation (sbert.net)

In [219]:
len(asquad_test_answers)+len(asquad_train_answers)+len(asquad_val_answers)

1504

In [220]:
print(sum(asquad_train_ids)==len(asquad_train_ids)*(len(asquad_train_ids)+1)/2)
print(sum(asquad_val_ids)==len(asquad_val_ids)*(len(asquad_val_ids)+1)/2)
print(sum(asquad_test_ids)==len(asquad_test_ids)*(len(asquad_test_ids)+1)/2)

True
True
True


In [221]:
def get_answered_feat(contexts, questions, answers, plausible):
    new_contexts, new_questions, new_answers = [], [], []
    for i in range(len(answers)):
        if plausible[i] == False:
            new_contexts.append(contexts[i])
            new_questions.append(questions[i])
            new_answers.append(answers[i])
    return new_contexts, new_questions, new_answers
span_train_contexts, span_train_questions, span_train_answers = get_answered_feat(asquad_train_contexts, asquad_train_questions, asquad_train_answers, asquad_train_plausible)   

In [222]:
print(len(span_train_contexts), len(asquad_train_contexts))
print(sum(asquad_test_plausible)) #del

902 902
0


In [223]:
#Creating the tokenizer
model_name =  "aubmindlab/araelectra-base-discriminator"
araelectra_tokenizer = AutoTokenizer.from_pretrained(model_name,do_lower_case=False)

In [224]:
span_train_encodings = araelectra_tokenizer(span_train_questions, span_train_contexts, truncation=True)
val_encodings = araelectra_tokenizer(asquad_val_questions, asquad_val_contexts, truncation=True, return_offsets_mapping=True)
test_encodings = araelectra_tokenizer(asquad_test_questions, asquad_test_contexts, truncation=True,  return_offsets_mapping=True)

why del cols?

In [225]:
val_offset = val_encodings['offset_mapping']
del val_encodings['offset_mapping']
test_offset = test_encodings['offset_mapping']
del test_encodings['offset_mapping']

In [226]:
val_ids_to_idx = {k:i for i,k in enumerate(asquad_val_ids)}
test_ids_to_idx = {k:i for i,k in enumerate(asquad_test_ids)}
print(val_ids_to_idx)

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31, 33: 32, 34: 33, 35: 34, 36: 35, 37: 36, 38: 37, 39: 38, 40: 39, 41: 40, 42: 41, 43: 42, 44: 43, 45: 44, 46: 45, 47: 46, 48: 47, 49: 48, 50: 49, 51: 50, 52: 51, 53: 52, 54: 53, 55: 54, 56: 55, 57: 56, 58: 57, 59: 58, 60: 59, 61: 60, 62: 61, 63: 62, 64: 63, 65: 64, 66: 65, 67: 66, 68: 67, 69: 68, 70: 69, 71: 70, 72: 71, 73: 72, 74: 73, 75: 74, 76: 75, 77: 76, 78: 77, 79: 78, 80: 79, 81: 80, 82: 81, 83: 82, 84: 83, 85: 84, 86: 85, 87: 86, 88: 87, 89: 88, 90: 89, 91: 90, 92: 91, 93: 92, 94: 93, 95: 94, 96: 95, 97: 96, 98: 97, 99: 98, 100: 99, 101: 100, 102: 101, 103: 102, 104: 103, 105: 104, 106: 105, 107: 106, 108: 107, 109: 108, 110: 109, 111: 110, 112: 111, 113: 112, 114: 113, 115: 114, 116: 115, 117: 116, 118: 117, 119: 118, 120: 119, 121: 120, 122: 12

In [227]:
def index_to_token_position(encodings , answers):
  start_positions = list()
  end_positions = list()
  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start'], 1))
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'], 1))
    #if context truncated
    if start_positions[-1] is None: 
      start_positions[-1] = araelectra_tokenizer.model_max_length
    #if end index is space
    itt = 1
    while end_positions[-1] is None: 
      end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end']-itt, 1)
      itt = itt + 1 
  encodings.update({'start_positions': torch.tensor(start_positions), 'end_positions': torch.tensor(end_positions)})
  encodings['start_positions'] = encodings['start_positions'].view(len(answers), 1)
  encodings['end_positions'] = encodings['end_positions'].view(len(answers), 1)

In [228]:
index_to_token_position(span_train_encodings, span_train_answers)
index_to_token_position(val_encodings, asquad_val_answers)
index_to_token_position(test_encodings, asquad_test_answers)

needed?

In [229]:
val_encodings['IDs'] = asquad_val_ids
test_encodings['IDs'] = asquad_test_ids

In [230]:
print(val_encodings.keys())
print(test_encodings.keys())
print(span_train_encodings.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions', 'IDs'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions', 'IDs'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])


In [231]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

In [232]:
class AqadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

span_train_dataset = AqadDataset(span_train_encodings)
val_dataset = AqadDataset(val_encodings)
test_dataset = AqadDataset(test_encodings)

In [233]:
data_collator = DataCollatorWithPadding(araelectra_tokenizer)

In [234]:
span_train_loader = DataLoader(span_train_dataset, batch_size=8, shuffle= True, collate_fn = data_collator)
val_loader = DataLoader(val_dataset, batch_size = 8, shuffle = True, collate_fn = data_collator)
test_loader = DataLoader(test_dataset, batch_size = 8, shuffle = True, collate_fn = data_collator)

In [235]:
def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best checkpoint
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [236]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to saved checkpoint
    model: model to load checkpoint parameters into       
    optimizer: optimizer defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    results = checkpoint['result_dict']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], results

In [237]:
def order_exp(base_path, exp_name):
  exp_path = os.path.join(base_path, exp_name)
  if not os.path.exists(exp_path):
    os.mkdir(exp_path)
  curr_ckp_path = os.path.join(exp_path,'curr.pt')
  best_ckp_path = os.path.join(exp_path, 'best.pt')
  return curr_ckp_path, best_ckp_path, exp_path

In [238]:
model_name1="ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA"

In [239]:
QA_AraElectra = ElectraForQuestionAnswering.from_pretrained(model_name1)

In [240]:
def freeze(Electra, count=None):
    if count is not None:
	      # We freeze here the embeddings of the model
        for param in Electra.embeddings.parameters():
            param.requires_grad = False

        if count != -1:
	          # if freeze_layer_count == -1, we only freeze the embedding layer
	          # otherwise we freeze the first `freeze_layer_count` encoder layers
            for layer in Electra.encoder.layer[:count]:
                for param in layer.parameters():
                    param.requires_grad = False
    print(sum(p.numel() for p in Electra.parameters()), sum(p.numel() for p in Electra.parameters() if p.requires_grad))

In [241]:
freeze(QA_AraElectra.electra, 6)

134602752 42527232


In [242]:
def get_raw_preds(data_loader, model,ids_to_index,offset,contexts, max_answer_length, n_best_size): 
  model.eval()
  imd_predictions,script_predictions = dict(), dict()
  with torch.no_grad():
    #F1 = EM = Total = 0
    total_loss = 0.0
    total_predictions = dict()
    no_probs_pred = dict()
    #loop = tqdm(data_loader)
    loop = tqdm(data_loader, leave=True)
    for batch_idx, batch in enumerate(loop):
      tokens = batch['input_ids'].to(device)
      masks = batch['attention_mask'].to(device)
      tokens_type = batch['token_type_ids'].to(device)
      gt_start = batch['start_positions'].to(device)
      gt_end = batch['end_positions'].to(device)
      IDs = batch['IDs'].to(device)
      outputs = model(tokens, masks, tokens_type, start_positions=gt_start, end_positions=gt_end)
      #calculating loss
      loss = outputs.loss
      #update average total loss 
      total_loss = total_loss + ((1 / (batch_idx + 1)) * (loss.item() - total_loss)) 
      #calculating f1 score and EM
      curr_batch_size = tokens.shape[0]
      post_raw_preds(IDs, outputs.start_logits, outputs.end_logits, ids_to_index, offset, contexts,max_answer_length, n_best_size, imd_predictions, script_predictions )
    #saving evaluation results
    #evaluation

    model.train()
    return imd_predictions,script_predictions

In [243]:
def post_raw_preds(IDs, total_start_logits, total_end_logits,ids_to_index,offset,contexts, max_answer_length, n_best_size,
 imd_predictions,script_predictions ):
    total_start_logits = total_start_logits.cpu().numpy()
    total_end_logits = total_end_logits.cpu().numpy()
    IDs = IDs.cpu().numpy()
    for i in range(IDs.shape[0]):
        offset_mapping = offset[ids_to_index[IDs[i].squeeze()]]
        # The first feature comes from the first example. For the more general case, we will need to be match the example_id to
        # an example index
        context = contexts[ids_to_index[IDs[i].squeeze()]]
        start_logits = total_start_logits[i]
        end_logits = total_end_logits[i]
        # Gather the indices the best start/end logits:
        start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
        end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
        valid_answers = []
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                # to part of the input_ids that are not in the context.
                if (
                    start_index >= len(offset_mapping)
                    or end_index >= len(offset_mapping)
                    or offset_mapping[start_index] is None
                    or offset_mapping[end_index] is None
                ):
                    continue
                # Don't consider answers with a length that is either < 0 or > max_answer_length.
                if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                    continue
                if start_index <= end_index: # We need to refine that test to check the answer is inside the context
                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        if len(valid_answers) ==0:
            valid_answers.append({"text":"", "score":""})

        valid_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        imd_predictions[str(IDs[i].squeeze())] = valid_answer
        script_predictions[str(IDs[i].squeeze())] = valid_answer['text']

In [244]:
def get_preds(total_preds, no_probs_preds,data_path, log_path):
    preds_path = os.path.join(log_path, 'preds')
    if not os.path.exists(preds_path):
        os.mkdir(preds_path)
    text_preds_path = os.path.join(preds_path, 'preds.json')
    jsonString = json.dumps(total_preds)
    jsonFile = open(text_preds_path, "w")
    jsonFile.write(jsonString)
    jsonFile.close()
    
    !python /content/Arabic-MRC/evaluatev2.py /content/asquadv2-val.json /content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth/preds/preds.json electra  --out-file /content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth
    if log_path:
        with open(os.path.join(log_path, 'res.csv')) as f:
            DictReader_obj = csv.DictReader(f)
            lastrow = None
            for item in DictReader_obj:
                lastrow = dict(item)
        #print(lastrow)
        return lastrow
    return 1

In [245]:
def span_train(model,start_epoch, num_epochs, optimizer,max_compined_metric, train_loader, val_loader, log, exp_name):
  curr_ckp_path, best_ckp_path, exp_path = order_exp('/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span', exp_name)
  model.train()
  for epoch in range(start_epoch,num_epochs):
    total_loss = 0.0
    loop = tqdm(train_loader, leave=True)
    for batch_idx, batch in enumerate(loop):
      tokens = batch['input_ids'].to(device)
      masks = batch['attention_mask'].to(device)
      tokens_type = batch['token_type_ids'].to(device)
      gt_start = batch['start_positions'].to(device)
      gt_end = batch['end_positions'].to(device)
      outputs = model(tokens, masks, tokens_type, start_positions=gt_start, end_positions=gt_end)
      loss = outputs.loss
      loss = 2*loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      total_loss = total_loss + ((1 / (batch_idx + 1)) * (loss.item() - total_loss)) 
      loop.set_description(f'Epoch {epoch}')
      loop.set_postfix(loss=loss.item())
    
    imd_preds, script_preds = get_raw_preds(val_loader, model,val_ids_to_idx,val_offset,asquad_val_contexts, 30, 10)
    result_dict = get_preds(script_preds, None,'/content/asquadv2-val.json',exp_path )
    checkpoint = {
            'epoch': epoch + 1,
            'result_dict':result_dict,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
    curr_compined_metric = float(result_dict['HasAns_exact'])+1.5*float(result_dict['HasAns_f1'])
    if curr_compined_metric>=max_compined_metric:
      max_compined_metric = curr_compined_metric
      save_ckp(checkpoint, True, curr_ckp_path, best_ckp_path)
    else:
      save_ckp(checkpoint, False, curr_ckp_path, best_ckp_path)
    print("ckp saved")
  return model

In [246]:
QA_AraElectra = ElectraForQuestionAnswering.from_pretrained(model_name1)
freeze(QA_AraElectra.electra, 6)
span_num_epochs = 4
span_learning_rate = 5e-5
span_optimizer = torch.optim.AdamW(QA_AraElectra.parameters(), lr=span_learning_rate, weight_decay=1e-5)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#criterion_span = nn.CrossEntropyLoss(reduction='none')
#cls_criterion = nn.CrossEntropyLoss()
QA_AraElectra.to(device)

134602752 42527232


ElectraForQuestionAnswering(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerN

why says 301 for val? where is train?

In [247]:
span_trained_model = span_train(QA_AraElectra,0, span_num_epochs, span_optimizer,0.0, span_train_loader, val_loader, True, '/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth')

100%|██████████| 38/38 [00:10<00:00,  3.50it/s]


{
  "exact": 35.548172757475086,
  "f1": 43.781352650418945,
  "total": 301,
  "HasAns_exact": 35.548172757475086,
  "HasAns_f1": 43.781352650418945,
  "HasAns_total": 301
}
ckp saved


100%|██████████| 38/38 [00:10<00:00,  3.48it/s]


{
  "exact": 38.205980066445186,
  "f1": 44.575480945816345,
  "total": 301,
  "HasAns_exact": 38.205980066445186,
  "HasAns_f1": 44.575480945816345,
  "HasAns_total": 301
}
ckp saved


100%|██████████| 38/38 [00:10<00:00,  3.49it/s]


{
  "exact": 38.205980066445186,
  "f1": 43.26380903649258,
  "total": 301,
  "HasAns_exact": 38.205980066445186,
  "HasAns_f1": 43.26380903649258,
  "HasAns_total": 301
}
ckp saved


100%|██████████| 38/38 [00:10<00:00,  3.49it/s]


{
  "exact": 39.8671096345515,
  "f1": 44.487283518712424,
  "total": 301,
  "HasAns_exact": 39.8671096345515,
  "HasAns_f1": 44.487283518712424,
  "HasAns_total": 301
}
ckp saved


In [248]:
qa_model = ElectraForQuestionAnswering.from_pretrained(model_name1)
span_learning_rate = 2e-5
span_optimizer = torch.optim.AdamW(qa_model.parameters(), lr=span_learning_rate, weight_decay=1e-5)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
qa_model.to(device)
qa_model , qa_optim, x, xx = load_ckp('/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth/best.pt', qa_model,span_optimizer)##why load ckp?

In [249]:
qa_model.save_pretrained('AraElectra-ASQuADv2-QA')##-finetuned-dataset
araelectra_tokenizer.save_pretrained('AraElectra-ASQuADv2-QA')##-finetuned-dataset

('AraElectra-ASQuADv2-QA/tokenizer_config.json',
 'AraElectra-ASQuADv2-QA/special_tokens_map.json',
 'AraElectra-ASQuADv2-QA/vocab.txt',
 'AraElectra-ASQuADv2-QA/added_tokens.json',
 'AraElectra-ASQuADv2-QA/tokenizer.json')

In [250]:
qa_model = ElectraForQuestionAnswering.from_pretrained('AraElectra-ASQuADv2-QA') ##modify to -finetuned-dataset?
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
qa_model.to(device)

ElectraForQuestionAnswering(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerN

In [251]:
imd_preds, script_preds = get_raw_preds(test_loader, qa_model,test_ids_to_idx,test_offset,asquad_test_contexts, 30, 10)

100%|██████████| 38/38 [00:10<00:00,  3.53it/s]


In [252]:
print(imd_preds)

{'52': {'score': 19.190504, 'text': 'مهمة واحدة فقط خلال مدة دراسته'}, '220': {'score': 8.223977, 'text': 'في جميع الفصول الدراسية التالية لاجتياز'}, '218': {'score': 12.757805, 'text': 'يسجل طالب الدكتوراة مقرر'}, '107': {'score': 16.165655, 'text': 'يجوز تمديدها لفصل دراسي واحد'}, '244': {'score': 8.687723, 'text': 'يسجل طالب الدكتوراة مقرر رسالة الدكتوراة في جميع الفصول'}, '179': {'score': 9.295003, 'text': 'يسجل طالب الدكتوراة مقرر رسالة الدكتوراة في جميع'}, '247': {'score': 8.026913, 'text': 'يسجل طالب الدكتوراة مقرر رسالة الدكتوراة في جميع الفصول'}, '110': {'score': 16.04367, 'text': 'هو 2 . 67'}, '57': {'score': 16.2567, 'text': 'الاشتراك وتذاكر سفر على الدرجة السياحية وبدل سفر يومي'}, '275': {'score': 16.948566, 'text': 'عن طريق الموقع الالكتروني لكلية الدراسات العليا'}, '13': {'score': 13.614878, 'text': 'عدد ساعات الدراسة خلال'}, '188': {'score': 9.3805065, 'text': 'يسجل طالب الدكتوراة مقرر رسالة الدكتوراة في جميع'}, '19': {'score': 10.163427, 'text': 'وتكون هذه المستندات مذك

In [253]:
def get_preds2(total_preds,data_path): 
  preds_path = os.path.join('preds')
  if not os.path.exists(preds_path):
    os.mkdir(preds_path)
  text_preds_path = os.path.join(preds_path, 'preds.json')
  jsonString = json.dumps(total_preds)
  jsonFile = open(text_preds_path, "w")
  jsonFile.write(jsonString)
  jsonFile.close()
  !python /content/Arabic-MRC/evaluatev2.py /content/asquadv2-test.json /content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth/preds/preds.json electra  

  ##return 1

In [254]:
qa_result_dict = get_preds2(script_preds, "/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth/preds" ) 
##'Data/asquadv2-test.json','Runs/AraElectraDecoupledAsquadv2/train/span/fourth/test-eval
### '/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth/preds' 

{
  "exact": 2.990033222591362,
  "f1": 6.300050253230052,
  "total": 301,
  "HasAns_exact": 2.990033222591362,
  "HasAns_f1": 6.300050253230052,
  "HasAns_total": 301
}


### **updates**

In [255]:
import os
import shutil
from collections import Counter
import numpy as np
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, ElectraForQuestionAnswering, DataCollatorWithPadding,BertModel, ElectraForSequenceClassification, ElectraModel
from arabert.preprocess import ArabertPreprocessor
import matplotlib.pyplot as plt
import seaborn as sns
import csv
torch.manual_seed(3407)

In [256]:
def add_end_index(answer, context):
  ## 1 if span match the context 0 otherwise
  text = answer['text']
  start_idx = answer['answer_start']
  end_idx = start_idx + len(text)
  answer['answer_end'] = end_idx
  if text == context[start_idx:end_idx]:
    answer['answer_end'] = end_idx
    return False
  for i in range(1,3):
    if text == context[start_idx-i:end_idx-i]:
      answer['answer_end']= end_idx-1
      answer['answer_start'] = start_idx-1
      return False
  return True

In [257]:
def arabert_preprocess(context,question, answer, arabert_prep):
    answer['text'] = arabert_prep.preprocess(answer['text'])
    context = arabert_prep.preprocess(context)
    question = arabert_prep.preprocess(question)
    res = context.find(answer['text'])
    if res !=-1:
        answer['answer_start'] = res
    return context, question, answer, res

In [258]:
def Read_Dataset(path,arabert_prep):
  contexts =[]
  answers =[]
  questions =[]
  IDs= []
 ## plausible = []
  cnt = 0
  with open(path) as f:
    aaqad_dict = json.load(f)
    for article in aaqad_dict['data']:
      for passage in article['paragraphs']:
        context = passage['context']
        for qa in passage['qas']:
          question = qa['question']
          '''if 'plausible_answers' in qa.keys():# there is two cases if the question have no answer then use plausible answer
            access = 'plausible_answers'
            plausible.append(True)
          else:
            access = 'answers'
            plausible.append(False)'''
          access = 'answers'
          for answer in qa[access]:
            context,question, answer, res =  arabert_preprocess(context,question, answer, arabert_prep)
            #if res==-1:
            #  cnt+=1
            #  continue
            flag = add_end_index(answer, context) #if false dont add the 
            cnt =cnt + flag
            flag = False
            if not flag:
              contexts.append(context)
              answers.append(answer)
              questions.append(question)
              IDs.append(int(qa['id']))
  return contexts,questions,answers,IDs #plausible

why fix them? can i directly use those ids and del id col in original csv?



In [259]:
def fix_ids(path):
  #IDs need to be fixed for evaluating purposes
    a_file = open(path, "r")
    json_object = json.load(a_file)
    a_file.close()
    idx_cnt = 1
    for article in json_object['data']:
      for passage in article['paragraphs']:
        context = passage['context']
        for qa in passage['qas']:
            qa['id'] = str(idx_cnt)
            idx_cnt = idx_cnt + 1
    a_file = open(path, "w")
    json.dump(json_object, a_file)
    a_file.close()

In [260]:
model_name = "araelectra-base-discriminator"
arabert_prep = ArabertPreprocessor(model_name=model_name)
fix_ids('/content/asquadv2-train.json')
fix_ids('/content/asquadv2-val.json')
fix_ids('/content/asquadv2-test.json')
asquad_span_train_contexts, asquad_span_train_questions, asquad_span_train_answers, asquad_span_train_ids = Read_Dataset('/content/asquadv2-train.json', arabert_prep)#,asquad_train_plausible
asquad_val_contexts, asquad_val_questions, asquad_val_answers, asquad_val_ids = Read_Dataset('/content/asquadv2-val.json', arabert_prep)#,asquad_val_plausible
asquad_test_contexts, asquad_test_questions, asquad_test_answers, asquad_test_ids = Read_Dataset('/content/asquadv2-test.json', arabert_prep)#,asquad_test_plausible

google Semantic Search — Sentence-Transformers documentation (sbert.net)

In [261]:
len(asquad_test_answers)+len(asquad_span_train_answers)+len(asquad_val_answers)

1504

In [262]:
print(sum(asquad_span_train_ids)==len(asquad_span_train_ids)*(len(asquad_span_train_ids)+1)/2)
print(sum(asquad_val_ids)==len(asquad_val_ids)*(len(asquad_val_ids)+1)/2)
print(sum(asquad_test_ids)==len(asquad_test_ids)*(len(asquad_test_ids)+1)/2)

True
True
True


In [263]:
def get_answered_feat(contexts, questions, answers):#, plausible
    new_contexts, new_questions, new_answers = [], [], []
    for i in range(len(answers)):
          new_contexts.append(contexts[i])
          new_questions.append(questions[i])
          new_answers.append(answers[i])
    return new_contexts, new_questions, new_answers
span_train_contexts, span_train_questions, span_train_answers = get_answered_feat(asquad_span_train_contexts, asquad_span_train_questions, asquad_span_train_answers)  ##, asquad_train_plausible 

In [264]:
print(len(span_train_contexts), len(asquad_span_train_contexts))

902 902


In [265]:
#Creating the tokenizer
model_name =  "aubmindlab/araelectra-base-discriminator"
araelectra_tokenizer = AutoTokenizer.from_pretrained(model_name,do_lower_case=False)

In [266]:
span_train_encodings = araelectra_tokenizer(span_train_questions, span_train_contexts, truncation=True, return_offsets_mapping=True)##remove , return_offsets_mapping=True
val_encodings = araelectra_tokenizer(asquad_val_questions, asquad_val_contexts, truncation=True, return_offsets_mapping=True)
test_encodings = araelectra_tokenizer(asquad_test_questions, asquad_test_contexts, truncation=True,  return_offsets_mapping=True)

why del cols?

In [267]:
span_train_offset = span_train_encodings['offset_mapping']
del span_train_encodings['offset_mapping']

val_offset = val_encodings['offset_mapping']
del val_encodings['offset_mapping']
test_offset = test_encodings['offset_mapping']
del test_encodings['offset_mapping']


In [268]:
val_ids_to_idx = {k:i for i,k in enumerate(asquad_val_ids)}
test_ids_to_idx = {k:i for i,k in enumerate(asquad_test_ids)}
span_train_ids_to_idx = {k:i for i,k in enumerate(asquad_span_train_ids)}

print(val_ids_to_idx)
print(test_ids_to_idx)
print(span_train_ids_to_idx)

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31, 33: 32, 34: 33, 35: 34, 36: 35, 37: 36, 38: 37, 39: 38, 40: 39, 41: 40, 42: 41, 43: 42, 44: 43, 45: 44, 46: 45, 47: 46, 48: 47, 49: 48, 50: 49, 51: 50, 52: 51, 53: 52, 54: 53, 55: 54, 56: 55, 57: 56, 58: 57, 59: 58, 60: 59, 61: 60, 62: 61, 63: 62, 64: 63, 65: 64, 66: 65, 67: 66, 68: 67, 69: 68, 70: 69, 71: 70, 72: 71, 73: 72, 74: 73, 75: 74, 76: 75, 77: 76, 78: 77, 79: 78, 80: 79, 81: 80, 82: 81, 83: 82, 84: 83, 85: 84, 86: 85, 87: 86, 88: 87, 89: 88, 90: 89, 91: 90, 92: 91, 93: 92, 94: 93, 95: 94, 96: 95, 97: 96, 98: 97, 99: 98, 100: 99, 101: 100, 102: 101, 103: 102, 104: 103, 105: 104, 106: 105, 107: 106, 108: 107, 109: 108, 110: 109, 111: 110, 112: 111, 113: 112, 114: 113, 115: 114, 116: 115, 117: 116, 118: 117, 119: 118, 120: 119, 121: 120, 122: 12

In [269]:
def index_to_token_position(encodings , answers):
  start_positions = list()
  end_positions = list()
  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start'], 1))
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'], 1))
    #if context truncated
    if start_positions[-1] is None: 
      start_positions[-1] = araelectra_tokenizer.model_max_length
    #if end index is space
    itt = 1
    while end_positions[-1] is None: 
      end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end']-itt, 1)
      itt = itt + 1 
  encodings.update({'start_positions': torch.tensor(start_positions), 'end_positions': torch.tensor(end_positions)})
  encodings['start_positions'] = encodings['start_positions'].view(len(answers), 1)
  encodings['end_positions'] = encodings['end_positions'].view(len(answers), 1)

In [270]:
index_to_token_position(span_train_encodings, span_train_answers)
index_to_token_position(val_encodings, asquad_val_answers)
index_to_token_position(test_encodings, asquad_test_answers)

needed?

In [271]:
val_encodings['IDs'] = asquad_val_ids
test_encodings['IDs'] = asquad_test_ids
span_train_encodings['IDs'] = asquad_span_train_ids

In [272]:
print(val_encodings.keys())
print(test_encodings.keys())
print(span_train_encodings.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions', 'IDs'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions', 'IDs'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions', 'IDs'])


In [273]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

In [274]:
class cgsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

span_train_dataset = cgsDataset(span_train_encodings)
val_dataset = cgsDataset(val_encodings)
test_dataset = cgsDataset(test_encodings)

In [275]:
data_collator = DataCollatorWithPadding(araelectra_tokenizer)

In [276]:
span_train_loader = DataLoader(span_train_dataset, batch_size=8, shuffle= True, collate_fn = data_collator)
val_loader = DataLoader(val_dataset, batch_size =8, shuffle = True, collate_fn = data_collator)
test_loader = DataLoader(test_dataset, batch_size = 8, shuffle = True, collate_fn = data_collator)

In [277]:
def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best checkpoint
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [278]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to saved checkpoint
    model: model to load checkpoint parameters into       
    optimizer: optimizer defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    results = checkpoint['result_dict']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], results

In [279]:
def order_exp(base_path, exp_name):
  exp_path = os.path.join(base_path, exp_name)
  if not os.path.exists(exp_path):
    os.mkdir(exp_path)
  curr_ckp_path = os.path.join(exp_path,'curr.pt')
  best_ckp_path = os.path.join(exp_path, 'best.pt')
  return curr_ckp_path, best_ckp_path, exp_path

In [280]:
model_name1="ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA"

In [281]:
QA_AraElectra = ElectraForQuestionAnswering.from_pretrained(model_name1)

In [282]:
def freeze(Electra, count=None):
    if count is not None:
	      # We freeze here the embeddings of the model
        for param in Electra.embeddings.parameters():
            param.requires_grad = False

        if count != -1:
	          # if freeze_layer_count == -1, we only freeze the embedding layer
	          # otherwise we freeze the first `freeze_layer_count` encoder layers
            for layer in Electra.encoder.layer[:count]:
                for param in layer.parameters():
                    param.requires_grad = False
    print(sum(p.numel() for p in Electra.parameters()), sum(p.numel() for p in Electra.parameters() if p.requires_grad))

In [283]:
freeze(QA_AraElectra.electra, 6)

134602752 42527232


In [284]:
def get_raw_preds(data_loader, model,ids_to_index,offset,contexts, max_answer_length, n_best_size): 
  model.eval()
  imd_predictions,script_predictions = dict(), dict()
  with torch.no_grad():
    #F1 = EM = Total = 0
    total_loss = 0.0
    total_predictions = dict()
    no_probs_pred = dict()
    #loop = tqdm(data_loader)
    loop = tqdm(data_loader, leave=True)
    for batch_idx, batch in enumerate(loop):
      tokens = batch['input_ids'].to(device)
      masks = batch['attention_mask'].to(device)
      tokens_type = batch['token_type_ids'].to(device)
      gt_start = batch['start_positions'].to(device)
      gt_end = batch['end_positions'].to(device)
      IDs = batch['IDs'].to(device)
      outputs = model(tokens, masks, tokens_type, start_positions=gt_start, end_positions=gt_end)
      #calculating loss
      loss = outputs.loss
      #update average total loss 
      total_loss = total_loss + ((1 / (batch_idx + 1)) * (loss.item() - total_loss)) 
      #calculating f1 score and EM
      curr_batch_size = tokens.shape[0]
      post_raw_preds(IDs, outputs.start_logits, outputs.end_logits, ids_to_index, offset, contexts,max_answer_length, n_best_size, imd_predictions, script_predictions )
    #saving evaluation results
    #evaluation

    model.train()
    return imd_predictions,script_predictions

In [285]:
def post_raw_preds(IDs, total_start_logits, total_end_logits,ids_to_index,offset,contexts, max_answer_length, n_best_size,
 imd_predictions,script_predictions ):
    total_start_logits = total_start_logits.cpu().numpy()
    total_end_logits = total_end_logits.cpu().numpy()
    IDs = IDs.cpu().numpy()
    for i in range(IDs.shape[0]):
        offset_mapping = offset[ids_to_index[IDs[i].squeeze()]]
        # The first feature comes from the first example. For the more general case, we will need to be match the example_id to
        # an example index
        context = contexts[ids_to_index[IDs[i].squeeze()]]
        start_logits = total_start_logits[i]
        end_logits = total_end_logits[i]
        # Gather the indices the best start/end logits:
        start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
        end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
        valid_answers = []
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                # to part of the input_ids that are not in the context.
                if (
                    start_index >= len(offset_mapping)
                    or end_index >= len(offset_mapping)
                    or offset_mapping[start_index] is None
                    or offset_mapping[end_index] is None
                ):
                    continue
                # Don't consider answers with a length that is either < 0 or > max_answer_length.
                if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                    continue
                if start_index <= end_index: # We need to refine that test to check the answer is inside the context
                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        if len(valid_answers) ==0:
            valid_answers.append({"text":"", "score":""})

        valid_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        imd_predictions[str(IDs[i].squeeze())] = valid_answer
        script_predictions[str(IDs[i].squeeze())] = valid_answer['text']

In [286]:
def get_preds(total_preds, no_probs_preds,data_path, log_path):
    preds_path = os.path.join(log_path, 'preds')
    if not os.path.exists(preds_path):
        os.mkdir(preds_path)
    text_preds_path = os.path.join(preds_path, 'preds.json')
    jsonString = json.dumps(total_preds)
    jsonFile = open(text_preds_path, "w")
    jsonFile.write(jsonString)
    jsonFile.close()
    
    !python /content/Arabic-MRC/evaluatev2.py /content/asquadv2-val.json /content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth/preds/preds.json electra  --out-file /content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth
    if log_path:
        with open(os.path.join(log_path, 'res.csv')) as f:
            DictReader_obj = csv.DictReader(f)
            lastrow = None
            for item in DictReader_obj:
                lastrow = dict(item)
        #print(lastrow)
        return lastrow
    return 1

In [287]:
def span_train(model,start_epoch, num_epochs, optimizer,max_compined_metric, train_loader, val_loader, log, exp_name):
  curr_ckp_path, best_ckp_path, exp_path = order_exp('/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span', exp_name)
  model.train()
  for epoch in range(start_epoch,num_epochs):
    total_loss = 0.0
    loop = tqdm(train_loader, leave=True)
    for batch_idx, batch in enumerate(loop):
      tokens = batch['input_ids'].to(device)
      masks = batch['attention_mask'].to(device)
      tokens_type = batch['token_type_ids'].to(device)
      gt_start = batch['start_positions'].to(device)
      gt_end = batch['end_positions'].to(device)
      outputs = model(tokens, masks, tokens_type, start_positions=gt_start, end_positions=gt_end)
      loss = outputs.loss
      loss = 2*loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      total_loss = total_loss + ((1 / (batch_idx + 1)) * (loss.item() - total_loss)) 
      loop.set_description(f'Epoch {epoch}')
      loop.set_postfix(loss=loss.item())
    
    imd_preds, script_preds = get_raw_preds(val_loader, model,val_ids_to_idx,val_offset,asquad_val_contexts, 30, 10)
    result_dict = get_preds(script_preds, None,'/content/asquadv2-val.json',exp_path )
    checkpoint = {
            'epoch': epoch + 1,
            'result_dict':result_dict,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
    curr_compined_metric = float(result_dict['HasAns_exact'])+1.5*float(result_dict['HasAns_f1'])
    if curr_compined_metric>=max_compined_metric:
      max_compined_metric = curr_compined_metric
      save_ckp(checkpoint, True, curr_ckp_path, best_ckp_path)
    else:
      save_ckp(checkpoint, False, curr_ckp_path, best_ckp_path)
    print("ckp saved")
  return model

In [288]:
QA_AraElectra = ElectraForQuestionAnswering.from_pretrained(model_name1)
freeze(QA_AraElectra.electra, 6)
span_num_epochs = 4
span_learning_rate = 2e-5
span_optimizer = torch.optim.AdamW(QA_AraElectra.parameters(), lr=span_learning_rate, weight_decay=1e-5)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#criterion_span = nn.CrossEntropyLoss(reduction='none')
#cls_criterion = nn.CrossEntropyLoss()
QA_AraElectra.to(device)

134602752 42527232


ElectraForQuestionAnswering(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerN

why says 301 for val? where is train?

In [289]:
span_trained_model = span_train(QA_AraElectra,0, span_num_epochs, span_optimizer,0.0, span_train_loader, val_loader, True, '/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth')

100%|██████████| 38/38 [00:11<00:00,  3.39it/s]


{
  "exact": 30.564784053156146,
  "f1": 43.48152589725722,
  "total": 301,
  "HasAns_exact": 30.564784053156146,
  "HasAns_f1": 43.48152589725722,
  "HasAns_total": 301
}
ckp saved


100%|██████████| 38/38 [00:10<00:00,  3.48it/s]


{
  "exact": 36.87707641196013,
  "f1": 44.22926166731683,
  "total": 301,
  "HasAns_exact": 36.87707641196013,
  "HasAns_f1": 44.22926166731683,
  "HasAns_total": 301
}
ckp saved


100%|██████████| 38/38 [00:10<00:00,  3.49it/s]


{
  "exact": 37.87375415282392,
  "f1": 44.24931179699838,
  "total": 301,
  "HasAns_exact": 37.87375415282392,
  "HasAns_f1": 44.24931179699838,
  "HasAns_total": 301
}
ckp saved


100%|██████████| 38/38 [00:10<00:00,  3.51it/s]


{
  "exact": 38.538205980066444,
  "f1": 44.6924495971346,
  "total": 301,
  "HasAns_exact": 38.538205980066444,
  "HasAns_f1": 44.6924495971346,
  "HasAns_total": 301
}
ckp saved


In [290]:
qa_model = ElectraForQuestionAnswering.from_pretrained(model_name1)
span_learning_rate = 2e-5
span_optimizer = torch.optim.AdamW(qa_model.parameters(), lr=span_learning_rate, weight_decay=1e-5)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
qa_model.to(device)
##qa_model , qa_optim, x, xx = load_ckp('/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth/best.pt', qa_model,span_optimizer)##why load ckp?

ElectraForQuestionAnswering(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerN

In [291]:
qa_model.save_pretrained('AraElectra-ASQuADv2-QA')##-finetuned-dataset
araelectra_tokenizer.save_pretrained('AraElectra-ASQuADv2-QA')##-finetuned-dataset

('AraElectra-ASQuADv2-QA/tokenizer_config.json',
 'AraElectra-ASQuADv2-QA/special_tokens_map.json',
 'AraElectra-ASQuADv2-QA/vocab.txt',
 'AraElectra-ASQuADv2-QA/added_tokens.json',
 'AraElectra-ASQuADv2-QA/tokenizer.json')

In [292]:
qa_model = ElectraForQuestionAnswering.from_pretrained('AraElectra-ASQuADv2-QA') ##modify to -finetuned-dataset?
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
qa_model.to(device)

ElectraForQuestionAnswering(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerN

In [293]:
imd_preds, script_preds = get_raw_preds(test_loader, qa_model,test_ids_to_idx,test_offset,asquad_test_contexts, 30, 10)

100%|██████████| 38/38 [00:10<00:00,  3.52it/s]


In [294]:
print(imd_preds)

{'52': {'score': 9.824321, 'text': 'واحدة'}, '220': {'score': 5.4642625, 'text': 'عقب او قرب انتهاء اجتياز جميع المقررات المحددة'}, '218': {'score': 0.85698354, 'text': '4 سنوات ويجوز تمديدها لسنتين اضافتين بموافقة كلية الدراسات العليا . الحد الزمني الاقصى لطالب الدكتوراة بدوام جزئي'}, '107': {'score': 5.2785273, 'text': 'ويجوز تمديدها لفصل دراسي واحد'}, '244': {'score': 4.5828457, 'text': 'وتسقط كسور الازمنة الاعتبارية'}, '179': {'score': 2.2761552, 'text': 'وتسقط كسور الازمنة الاعتبارية'}, '247': {'score': 6.0235167, 'text': 'كسور الازمنة الاعتبارية'}, '110': {'score': 10.623768, 'text': '2 . 67'}, '57': {'score': 11.660757, 'text': 'نفقات الاشتراك وتذاكر سفر على الدرجة السياحية وبدل سفر يومي قدره 30 دينار كويتي'}, '275': {'score': 15.670231, 'text': 'الموقع الالكتروني لكلية الدراسات العليا'}, '13': {'score': 2.6145058, 'text': 'شروط التقديم على كلية الدراسات العليا متغيرة'}, '188': {'score': 3.8604045, 'text': 'وتسقط كسور الازمنة الاعتبارية'}, '19': {'score': 10.760812, 'text': 'صور

In [295]:
def get_preds2(total_preds,data_path): 
  preds_path = os.path.join('preds')
  if not os.path.exists(preds_path):
    os.mkdir(preds_path)
  text_preds_path = os.path.join(preds_path, 'preds.json')
  jsonString = json.dumps(total_preds)
  jsonFile = open(text_preds_path, "w")
  jsonFile.write(jsonString)
  jsonFile.close()
  !python /content/Arabic-MRC/evaluatev2.py /content/asquadv2-test.json /content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth/preds/preds.json electra  

  ##return 1

In [296]:
qa_result_dict = get_preds2(script_preds, "/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth/preds" ) 
##'Data/asquadv2-test.json','Runs/AraElectraDecoupledAsquadv2/train/span/fourth/test-eval
### '/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth/preds' 

{
  "exact": 2.990033222591362,
  "f1": 6.1972921266246885,
  "total": 301,
  "HasAns_exact": 2.990033222591362,
  "HasAns_f1": 6.1972921266246885,
  "HasAns_total": 301
}


In [297]:
print(df.iloc[0])

print(df.iloc[104])


title                                   تنظيم كلية الدراسات العليا
context          رسالة كلية الدراسات العليا هي العمل المخطط اله...
question                         ما هي رسالة كلية الدراسات العليا؟
answer           العمل المخطط الهادف الى المساهمة في تنمية إمكا...
answer_start                                                    30
is_impossible                                                False
count                                                            1
ID                                        56be85543aeaaa14008c9063
Name: 0, dtype: object
title                                               درجة الماجستير
context          شروط القبول النظامي للماجستير هي الحصول على در...
question                     ما هي موانع القبول النظامي للماجستير؟
answer           الفصل الأكاديمي أو حصول المتقدم على دبلوم دراس...
answer_start                                                   251
is_impossible                                                False
count                                  

### **Useful tools** (IGNORE)

**Zeyad Ahmed recommendations**
 https://huggingface.co/ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA 
In part a:
- you are loading the model into something called huggingface pipeline (essentially it's like the whole process of tokenization and passing this tokenization to the model and preprocess the output to get the answer)
-So based on the previous declaration you can't fine tune the model through the pipeline.

in part b)
-I'm showing how can you load the model only so you can treat it as a normal pytorch model and finetune it or use it as you want
-So part b does not affect the process at all it's just an extra reference to how to load and use the model separately.


**Using Zeyad Ahmed Script for Finetuning**

1-To change you file from csv format to json format I have the dataset initially as csv format and changed to json using this script https://github.com/zeyadahmed10/Arabic-MRC/blob/main/Translator/translation2dataset.py my csv obviously differs from you but it can guide you

2-regarding the finetuing this notebook https://github.com/zeyadahmed10/Arabic-MRC/blob/main/AraElectraDecoupling-ASQuADv2.ipynb will help you it includes the whole process of data processing, checkpoint loading , training and testing

3-you can have any number of questions for a one context as you want no maximum limit

4-you can create the missing fields in the csv in the fly like answer_start, id and any some fields will not contribute to the training you can make it as null refer to the notebook in (2) to know more about the training

In [ ]:
###Direct usage of zeyads code
from transformers import ElectraForQuestionAnswering, ElectraForSequenceClassification, AutoTokenizer, pipeline
from arabert.preprocess import ArabertPreprocessor
#only preprocessing
model_name="aubmindlab/araelectra-base-discriminator"
prep_object = ArabertPreprocessor(model_name=model_name)
question = prep_object.preprocess('ما هو المعدل المطلوب للطالب الغير مقيد؟')
context = prep_object.preprocess('''

المعدل المطلوب للطالب المقيد هو 2.67. المعدل المطلوب للطالب الغير مقيد هو 2.5. من شروط التقديم على الدراسات العليا هو شهادة التخرج وشهادة اللغة وكشف الدرجات.

''')
# a) Get predictions using his finetuned model
#solution: train his model using the script?
qa_modelname = '/content/AraElectra-ASQuADv2-QA'
##cls_modelname = '/content/AraElectra-ASQuADv2-CLS'
qa_pipe = pipeline('question-answering', model=qa_modelname, tokenizer=qa_modelname)
##cls_pipe = pipeline('question-answering', model=cls_modelname, tokenizer=cls_modelname)
QA_input = {
    'question': question,
    'context': context
}
'''
CLS_input = {
    'question': question,
    'context': context
}'''
##print(CLS_input)
qa_res = qa_pipe(QA_input)
##cls_res = cls_pipe(CLS_input)
##print(CLS_input)
threshold = 0.5 #hyperparameter can be tweaked
## note classification results label0 probability it can be answered label1 probability can't be answered 
## if label1 probability > threshold then consider the output of qa_res is empty string else take the qa_res
# b) Load model & tokenizer
'''
qa_model = ElectraForQuestionAnswering.from_pretrained(qa_modelname)
cls_model = ElectraForSequenceClassification.from_pretrained(cls_modelname)
tokenizer = AutoTokenizer.from_pretrained(qa_modelname)'''
print('here')
print(qa_res)
##print(cls_res)

here
{'score': 0.27281802892684937, 'start': 41, 'end': 82, 'answer': 'المعدل المطلوب للطالب الغير مقيد هو 2 . 5'}


In [ ]:
from arabert.preprocess import ArabertPreprocessor

model_name = "araelectra-base-discriminator"
arabert_prep = ArabertPreprocessor(model_name=model_name)

text = "ولن نبالغ إذا قلنا: إن 'هاتف' أو 'كمبيوتر المكتب' في زمننا هذا ضروري"
arabert_prep.unpreprocess(text)

"ولن نبالغ إذا قلنا: إن 'هاتف'أو'كمبيوتر المكتب' في زمننا هذا ضروري"

In [ ]:
from arabert.preprocess import ArabertPreprocessor

model_name="araelectra-base-discriminator"
arabert_prep = ArabertPreprocessor(model_name=model_name)

text = "ولن نبالغ إذا قلنا: إن 'هاتف' أو 'كمبيوتر المكتب' في زمننا هذا ضروري"
arabert_prep.preprocess(text)

"ولن نبالغ إذا قلنا : إن ' هاتف ' أو ' كمبيوتر المكتب ' في زمننا هذا ضروري"

In [ ]:
#to check zeyad dataset is actually being read by importing it from hf
import datasets

dataset = datasets.load_dataset(
  'FatemahAlsubaiei/mydata', split='train'
)
print(dataset[1],"here1")
print(dataset['answer'],"here2")
print("here3")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/FatemahAlsubaiei___csv/FatemahAlsubaiei--mydata-9ebc89f6a833abe9/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.
{'title': 'بيونسيه', 'context': 'بيونسيه جيزيل نولز-كارتر (بالإنجليزية: Beyoncé Giselle Knowles-Carter) هي مغنية وكاتبة أغاني ومنتجة أسطوانات وممثلة أمريكية، ولدت في 4 سبتمبر 1981. ولدت وترعرعت في هيوستن ، تكساس ، وأدت في العديد من مسابقات الغناء والرقص عندما كانت طفلة ، وارتفعت إلى الشهرة في أواخر 1990s كمغنية رئيسية في مجموعة R & B للفتيات Destiny\'s Child. شهدت فترة توقفهما إصدار ألبوم بيونسيه الأول ، Dangerously in Love (2003) ، الذي أسسها كفنانة منفردة في جميع أنحاء العالم ، وحصل على خمس جوائز غرامي وعرض بيلبورد هوت 100 أغنية منفردة رقم واحد "Crazy in Love" و "Baby Boy".', 'question': 'ما هي المجالات التي تنافست فيها بيونسيه عندما كانت تكبر؟', 'answer': 'الغناء والرقص', 'answer_start': 207, 'is_impossible': False, 'count': 2, 'ID': '56b

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### **Running Zeyad's finetuning script on my own dataset - (WITH CLASSIFICATION)** NEGLECT THIS PART

In [ ]:
#@title
import os
import shutil
from collections import Counter
import numpy as np
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, ElectraForQuestionAnswering, DataCollatorWithPadding,BertModel, ElectraForSequenceClassification, ElectraModel
from arabert.preprocess import ArabertPreprocessor
import matplotlib.pyplot as plt
import seaborn as sns
import csv
torch.manual_seed(3407)

In [ ]:
#@title
def add_end_index(answer, context):
  ## 1 if span match the context 0 otherwise
  text = answer['text']
  start_idx = answer['answer_start']
  end_idx = start_idx + len(text)
  answer['answer_end'] = end_idx
  if text == context[start_idx:end_idx]:
    answer['answer_end'] = end_idx
    return False
  for i in range(1,3):
    if text == context[start_idx-i:end_idx-i]:
      answer['answer_end']= end_idx-1
      answer['answer_start'] = start_idx-1
      return False
  return True

In [ ]:
#@title
def arabert_preprocess(context,question, answer, arabert_prep):
    answer['text'] = arabert_prep.preprocess(answer['text'])
    context = arabert_prep.preprocess(context)
    question = arabert_prep.preprocess(question)
    res = context.find(answer['text'])
    if res !=-1:
        answer['answer_start'] = res
    return context, question, answer, res

In [ ]:
#@title
def Read_AAQAD(path,arabert_prep):
  contexts =[]
  answers =[]
  questions =[]
  IDs= []
  plausible = []
  cnt = 0
  with open(path) as f:
    aaqad_dict = json.load(f)
    for article in aaqad_dict['data']:
      for passage in article['paragraphs']:
        context = passage['context']
        for qa in passage['qas']:
          question = qa['question']
          if 'plausible_answers' in qa.keys():# there is two cases if the question have no answer then use plausible answer
            access = 'plausible_answers'
            plausible.append(True)
          else:
            access = 'answers'
            plausible.append(False)
          for answer in qa[access]:
            context,question, answer, res =  arabert_preprocess(context,question, answer, arabert_prep)
            #if res==-1:
            #  cnt+=1
            #  continue
            flag = add_end_index(answer, context) #if false dont add the 
            cnt =cnt + flag
            flag = False
            if not flag:
              contexts.append(context)
              answers.append(answer)
              questions.append(question)
              IDs.append(int(qa['id']))
  return contexts,questions,answers,plausible,IDs

In [ ]:
#@title
def fix_ids(path):
  #IDs need to be fixed for evaluating purposes
    a_file = open(path, "r")
    json_object = json.load(a_file)
    a_file.close()
    idx_cnt = 1
    for article in json_object['data']:
      for passage in article['paragraphs']:
        context = passage['context']
        for qa in passage['qas']:
            qa['id'] = str(idx_cnt)
            idx_cnt = idx_cnt + 1
    a_file = open(path, "w")
    json.dump(json_object, a_file)
    a_file.close()

In [ ]:
#@title
model_name = "araelectra-base-discriminator"
arabert_prep = ArabertPreprocessor(model_name=model_name)
fix_ids('/content/asquadv2-train.json')
fix_ids('/content/asquadv2-val.json')
fix_ids('/content/asquadv2-test.json')
asquad_train_contexts, asquad_train_questions, asquad_train_answers,asquad_train_plausible, asquad_train_ids = Read_AAQAD('/content/asquadv2-train.json', arabert_prep)
asquad_val_contexts, asquad_val_questions, asquad_val_answers,asquad_val_plausible, asquad_val_ids = Read_AAQAD('/content/asquadv2-val.json', arabert_prep)
asquad_test_contexts, asquad_test_questions, asquad_test_answers,asquad_test_plausible, asquad_test_ids = Read_AAQAD('/content/asquadv2-test.json', arabert_prep)

In [ ]:
#@title
len(asquad_test_answers)+len(asquad_train_answers)+len(asquad_val_answers)

60

In [ ]:
#@title
print(len(asquad_test_answers))
print(len(asquad_val_answers))
print(len(asquad_train_answers))

6
6
48


In [ ]:
#@title
sum(asquad_test_plausible)/len(asquad_test_plausible) #shows the percentage of not answerd questions
#zero because i have no unanswerable questions

0.0

In [ ]:
#@title
print(sum(asquad_train_ids)==len(asquad_train_ids)*(len(asquad_train_ids)+1)/2)
print(sum(asquad_val_ids)==len(asquad_val_ids)*(len(asquad_val_ids)+1)/2)
print(sum(asquad_test_ids)==len(asquad_test_ids)*(len(asquad_test_ids)+1)/2)

True
True
True


In [ ]:
#@title
def get_answered_feat(contexts, questions, answers, plausible):
    new_contexts, new_questions, new_answers = [], [], []
    for i in range(len(answers)):
        if plausible[i] == False:
            new_contexts.append(contexts[i])
            new_questions.append(questions[i])
            new_answers.append(answers[i])
    return new_contexts, new_questions, new_answers
span_train_contexts, span_train_questions, span_train_answers = get_answered_feat(asquad_train_contexts, asquad_train_questions, asquad_train_answers, asquad_train_plausible)   

In [ ]:
#@title
print(len(span_train_contexts), len(asquad_train_contexts))
print(sum(asquad_test_plausible))

48 48
0


In [ ]:
#@title
#Creating the tokenizer
model_name =  "aubmindlab/araelectra-base-discriminator"
araelectra_tokenizer = AutoTokenizer.from_pretrained(model_name,do_lower_case=False)

In [ ]:
#@title
train_encodings = araelectra_tokenizer(asquad_train_questions, asquad_train_contexts, truncation = True )
span_train_encodings = araelectra_tokenizer(span_train_questions, span_train_contexts, truncation=True)
val_encodings = araelectra_tokenizer(asquad_val_questions, asquad_val_contexts, truncation=True, return_offsets_mapping=True)
test_encodings = araelectra_tokenizer(asquad_test_questions, asquad_test_contexts, truncation=True,  return_offsets_mapping=True)

In [ ]:
#@title
val_offset = val_encodings['offset_mapping']
del val_encodings['offset_mapping']
test_offset = test_encodings['offset_mapping']
del test_encodings['offset_mapping']

In [ ]:
#@title
val_ids_to_idx = {k:i for i,k in enumerate(asquad_val_ids)}
test_ids_to_idx = {k:i for i,k in enumerate(asquad_test_ids)}
print(val_ids_to_idx)

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5}


In [ ]:
#@title
def index_to_token_position(encodings , answers):
  start_positions = list()
  end_positions = list()
  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start'], 1))
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'], 1))
    #if context truncated
    if start_positions[-1] is None: 
      start_positions[-1] = araelectra_tokenizer.model_max_length
    #if end index is space
    itt = 1
    while end_positions[-1] is None: 
      end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end']-itt, 1)
      itt = itt + 1 
  encodings.update({'start_positions': torch.tensor(start_positions), 'end_positions': torch.tensor(end_positions)})
  encodings['start_positions'] = encodings['start_positions'].view(len(answers), 1)
  encodings['end_positions'] = encodings['end_positions'].view(len(answers), 1)

In [ ]:
#@title
index_to_token_position(span_train_encodings, span_train_answers)
index_to_token_position(val_encodings, asquad_val_answers)
index_to_token_position(test_encodings, asquad_test_answers)

not needed? because i will remove CLS parts?

In [ ]:
#@title
def add_weights_labels_tensors(encodings, plausible):
  plausible = torch.tensor(plausible)
  weights = torch.ones(plausible.shape)
  no_ans = torch.ones(plausible.shape)
  weights[plausible==False]=2.0
  no_ans[plausible==False]=0.0
  weights = weights.view(-1,1)
  no_ans = no_ans.view(-1,1)
  encodings.update({'weights':weights, 'no_ans':no_ans})

In [ ]:
#@title
add_weights_labels_tensors(train_encodings, asquad_train_plausible)
add_weights_labels_tensors(val_encodings, asquad_val_plausible)
add_weights_labels_tensors(test_encodings, asquad_test_plausible)

In [ ]:
#@title
val_encodings['IDs'] = asquad_val_ids
test_encodings['IDs'] = asquad_test_ids

In [ ]:
#@title
print(train_encodings.keys())
print(val_encodings.keys())
print(test_encodings.keys())
print(span_train_encodings.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'weights', 'no_ans'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions', 'weights', 'no_ans', 'IDs'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions', 'weights', 'no_ans', 'IDs'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])


In [ ]:
#@title
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
#@title
class AqadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

cls_train_dataset = AqadDataset(train_encodings)
span_train_dataset = AqadDataset(span_train_encodings)
val_dataset = AqadDataset(val_encodings)
test_dataset = AqadDataset(test_encodings)

In [ ]:
#@title
data_collator = DataCollatorWithPadding(araelectra_tokenizer)

In [ ]:
#@title
cls_train_loader = DataLoader(cls_train_dataset, batch_size=8, shuffle= True, collate_fn= data_collator)
span_train_loader = DataLoader(span_train_dataset, batch_size=8, shuffle= True, collate_fn = data_collator)
val_loader = DataLoader(val_dataset, batch_size = 8, shuffle = True, collate_fn = data_collator)
test_loader = DataLoader(test_dataset, batch_size = 8, shuffle = True, collate_fn = data_collator)

In [ ]:
#@title
def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best checkpoint
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
#@title
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to saved checkpoint
    model: model to load checkpoint parameters into       
    optimizer: optimizer defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    results = checkpoint['result_dict']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], results

In [ ]:
#@title
def order_exp(base_path, exp_name):
  exp_path = os.path.join(base_path, exp_name)
  if not os.path.exists(exp_path):
    os.mkdir(exp_path)
  curr_ckp_path = os.path.join(exp_path,'curr.pt')
  best_ckp_path = os.path.join(exp_path, 'best.pt')
  return curr_ckp_path, best_ckp_path, exp_path

In [ ]:
#@title
def cls_eval(model, data_loader, exp_path, train_loss):
    model.eval()
    total_acc = 0
    total_pred = None
    total_IDs = None
    soft = torch.nn.Softmax(dim=1)
    with open(os.path.join(exp_path,'preds.txt'),'w') as f:
        pass
    for batch in data_loader:
      tokens = batch['input_ids'].to(device)
      masks = batch['attention_mask'].to(device)
      tokens_type = batch['token_type_ids'].to(device)
      IDs = batch['IDs'].to(device)
      gt_no_ans = batch['no_ans'].to(device)
      output = model(tokens, masks, tokens_type)
      pred = output.logits.view(masks.shape[0],2,)
      with open(os.path.join(exp_path,'preds.txt'),'a') as f:
        preds_soft = soft(pred)[:,1]
        IDs
        for i in range(preds_soft.shape[0]):
            f.write(f"{IDs[i].item()},{preds_soft[i].item()}\n")
        
      pred = torch.argmax(pred, dim=1)
      target = batch['no_ans'].to(device).view(masks.shape[0],)
      total_acc += torch.sum(target==pred)

    total_acc = total_acc/ val_dataset.__len__()
    res_dict = {'acc':total_acc.item()*100, 'train_loss':train_loss}
    if exp_path:
        log_path = os.path.join(exp_path,'res.csv')
        if not os.path.exists(log_path):
            with open(log_path,'w') as f:
                writer = csv.DictWriter(f, fieldnames=res_dict.keys())
                writer.writeheader()
        with open(log_path, 'a') as f:
            writer = csv.DictWriter(f, fieldnames=res_dict.keys())
            #writer.writeheader()
            writer.writerow(res_dict)
    model.train()
    return res_dict

In [ ]:
#@title
def cls_train(model,start_epoch, num_epochs, optimizer,max_acc, train_loader, val_loader, log, exp_name):
  curr_ckp_path, best_ckp_path, exp_path = order_exp('/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/cls', exp_name)
  model.train()
  cls_pred = None
  for epoch in range(start_epoch,num_epochs):
    total_loss = 0.0
    loop = tqdm(train_loader, leave=True)
    for batch_idx, batch in enumerate(loop):
      tokens = batch['input_ids'].to(device)
      masks = batch['attention_mask'].to(device)
      tokens_type = batch['token_type_ids'].to(device)
      weights = batch['weights'].to(device)
      output = model(tokens, masks, tokens_type)
      pred = output.logits.view(masks.shape[0],2,)
      target = batch['no_ans'].type(torch.LongTensor)
      target = target.to(device)
      loss = cls_criterion(pred, target.view(masks.shape[0],))
      loss = loss*(weights.view(masks.shape[0],))
      loss = torch.mean(loss)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      total_loss = total_loss + ((1 / (batch_idx + 1)) * (loss.item() - total_loss)) 
      loop.set_description(f'Epoch {epoch}')
      loop.set_postfix(loss=loss.item())

    result_dict= cls_eval(model, val_loader,exp_path,total_loss )
    checkpoint = {
            'epoch': epoch + 1,
            'result_dict':result_dict,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
    curr_acc = result_dict['acc']
    if curr_acc>=max_acc:
      max_acc = curr_acc
      save_ckp(checkpoint, True, curr_ckp_path, best_ckp_path)
    else:
      save_ckp(checkpoint, False, curr_ckp_path, best_ckp_path)
    print(result_dict)
  return model

In [ ]:
#@title
model_namex="ZeyadAhmed/AraElectra-Arabic-SQuADv2-CLS"
model_namexx="ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA"

In [ ]:
#@title
Cls_AraElectra = ElectraForSequenceClassification.from_pretrained(model_namex, num_labels=2)
QA_AraElectra = ElectraForQuestionAnswering.from_pretrained(model_namexx)

why use freeze and not .from_pretrained() only?

In [ ]:
#@title
def freeze(Electra, count=None):
    if count is not None:
	      # We freeze here the embeddings of the model
        for param in Electra.embeddings.parameters():
            param.requires_grad = False

        if count != -1:
	          # if freeze_layer_count == -1, we only freeze the embedding layer
	          # otherwise we freeze the first `freeze_layer_count` encoder layers
            for layer in Electra.encoder.layer[:count]:
                for param in layer.parameters():
                    param.requires_grad = False
    print(sum(p.numel() for p in Electra.parameters()), sum(p.numel() for p in Electra.parameters() if p.requires_grad))

In [ ]:
#@title
freeze(Cls_AraElectra.electra,6)
freeze(QA_AraElectra.electra, 6)

134602752 42527232
134602752 42527232


In [ ]:
#@title
num_epochs = 2
learning_rate = 3e-5
optimizer = torch.optim.Adam(Cls_AraElectra.parameters(), lr=learning_rate)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#criterion_span = nn.CrossEntropyLoss(reduction='none')
cls_criterion = nn.CrossEntropyLoss(reduction='none')
Cls_AraElectra.to(device)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [ ]:
#@title
cls_trained_model= cls_train(Cls_AraElectra, 0, 2, optimizer, 0, cls_train_loader, val_loader , True, '/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/cls/third')

Epoch 0: 100%|██████████| 6/6 [00:05<00:00,  1.02it/s, loss=0.0737]


{'acc': 100.0, 'train_loss': 0.5758701711893082}


Epoch 1: 100%|██████████| 6/6 [00:02<00:00,  2.04it/s, loss=0.00716]


{'acc': 100.0, 'train_loss': 0.015093949235354861}


In [ ]:
#@title
cls_trained_model = cls_train(cls_trained_model, 2, 6, optimizer, 84.3, cls_train_loader, val_loader , True, '/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/cls/third')

Epoch 2: 100%|██████████| 6/6 [00:02<00:00,  2.07it/s, loss=0.00382]


{'acc': 100.0, 'train_loss': 0.0053530742491905885}


Epoch 3: 100%|██████████| 6/6 [00:02<00:00,  2.04it/s, loss=0.00228]


{'acc': 100.0, 'train_loss': 0.0031442715165515738}


Epoch 4: 100%|██████████| 6/6 [00:02<00:00,  2.04it/s, loss=0.00219]


{'acc': 100.0, 'train_loss': 0.0022031382735197744}


Epoch 5: 100%|██████████| 6/6 [00:02<00:00,  2.05it/s, loss=0.00145]


{'acc': 100.0, 'train_loss': 0.001767653040587902}


In [ ]:
#@title
cls_trained_model, optimizer , x, xx = load_ckp('/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/cls/third/curr.pt',Cls_AraElectra, optimizer)

In [ ]:
#@title
cls_trained_model = cls_train(cls_trained_model, 3, 8, optimizer, 85.1, cls_train_loader, val_loader , True, '/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/cls/third')

Epoch 3: 100%|██████████| 6/6 [00:02<00:00,  2.01it/s, loss=0.00133]


{'acc': 100.0, 'train_loss': 0.0016433807516780992}


Epoch 4: 100%|██████████| 6/6 [00:02<00:00,  2.02it/s, loss=0.00103]


{'acc': 100.0, 'train_loss': 0.0012254234946643312}


Epoch 5: 100%|██████████| 6/6 [00:02<00:00,  2.00it/s, loss=0.00104]


{'acc': 100.0, 'train_loss': 0.0012242689457101126}


Epoch 6: 100%|██████████| 6/6 [00:02<00:00,  2.01it/s, loss=0.00127]


{'acc': 100.0, 'train_loss': 0.0010488863529947896}


Epoch 7: 100%|██████████| 6/6 [00:03<00:00,  1.98it/s, loss=0.000906]


{'acc': 100.0, 'train_loss': 0.0009476618918900688}


In [ ]:
#@title
cls_model = ElectraForSequenceClassification.from_pretrained(model_namex)

In [ ]:
#@title
learning_rate = 3e-5
optimizer = torch.optim.AdamW(cls_model.parameters(), lr=learning_rate)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#criterion_span = nn.CrossEntropyLoss(reduction='none')
#cls_criterion = nn.CrossEntropyLoss()
cls_model.to(device)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [ ]:
#@title
cls_model, optimizer, start_epoch, result_dict = load_ckp('/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/cls/third/best.pt', cls_model, optimizer)

In [ ]:
#@title
def get_raw_preds(data_loader, model,ids_to_index,offset,contexts, max_answer_length, n_best_size): 
  model.eval()
  imd_predictions,script_predictions = dict(), dict()
  with torch.no_grad():
    #F1 = EM = Total = 0
    total_loss = 0.0
    total_predictions = dict()
    no_probs_pred = dict()
    #loop = tqdm(data_loader)
    loop = tqdm(data_loader, leave=True)
    for batch_idx, batch in enumerate(loop):
      tokens = batch['input_ids'].to(device)
      masks = batch['attention_mask'].to(device)
      tokens_type = batch['token_type_ids'].to(device)
      gt_start = batch['start_positions'].to(device)
      gt_end = batch['end_positions'].to(device)
      IDs = batch['IDs'].to(device)
      outputs = model(tokens, masks, tokens_type, start_positions=gt_start, end_positions=gt_end)
      #calculating loss
      loss = outputs.loss
      #update average total loss 
      total_loss = total_loss + ((1 / (batch_idx + 1)) * (loss.item() - total_loss)) 
      #calculating f1 score and EM
      curr_batch_size = tokens.shape[0]
      post_raw_preds(IDs, outputs.start_logits, outputs.end_logits, ids_to_index, offset, contexts,max_answer_length, n_best_size, imd_predictions, script_predictions )
    #saving evaluation results
    #evaluation

    model.train()
    return imd_predictions,script_predictions

here to rexplain

In [ ]:
#@title
def post_raw_preds(IDs, total_start_logits, total_end_logits,ids_to_index,offset,contexts, max_answer_length, n_best_size,
 imd_predictions,script_predictions ):
    total_start_logits = total_start_logits.cpu().numpy()
    total_end_logits = total_end_logits.cpu().numpy()
    IDs = IDs.cpu().numpy()
    for i in range(IDs.shape[0]):
        offset_mapping = offset[ids_to_index[IDs[i].squeeze()]]
        # The first feature comes from the first example. For the more general case, we will need to be match the example_id to
        # an example index
        context = contexts[ids_to_index[IDs[i].squeeze()]]
        start_logits = total_start_logits[i]
        end_logits = total_end_logits[i]
        # Gather the indices the best start/end logits:
        start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
        end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
        valid_answers = []
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                # to part of the input_ids that are not in the context.
                if (
                    start_index >= len(offset_mapping)
                    or end_index >= len(offset_mapping)
                    or offset_mapping[start_index] is None
                    or offset_mapping[end_index] is None
                ):
                    continue
                # Don't consider answers with a length that is either < 0 or > max_answer_length.
                if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                    continue
                if start_index <= end_index: # We need to refine that test to check the answer is inside the context
                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        if len(valid_answers) ==0:
            valid_answers.append({"text":"", "score":""})

        valid_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        imd_predictions[str(IDs[i].squeeze())] = valid_answer
        script_predictions[str(IDs[i].squeeze())] = valid_answer['text']

In [ ]:
#@title
def get_preds(total_preds, no_probs_preds,data_path, log_path):
    preds_path = os.path.join(log_path, 'preds')
    if not os.path.exists(preds_path):
        os.mkdir(preds_path)
    no_probs_path = os.path.join(preds_path, 'na_probs.json')
    text_preds_path = os.path.join(preds_path, 'preds.json')
    jsonString = json.dumps(total_preds)
    jsonFile = open(text_preds_path, "w")
    jsonFile.write(jsonString)
    jsonFile.close()
    if no_probs_preds is not None:
        jsonString = json.dumps(no_probs_preds)
        jsonFile = open(no_probs_path, "w")
        jsonFile.write(jsonString)
        jsonFile.close()
        #!python evaluatev2.py data_path text_preds_path electra --na-prob-file no_probs_path --na-prob-thresh 0.4 --out-file log_path
        #os.system(f"python evaluatev2.py {data_path} {text_preds_path} electra --na-prob-file {no_probs_path} --na-prob-thresh 0.5 --out-file {log_path}")
        !python /content/Arabic-MRC/evaluatev2.py /content/asquadv2-val.json /content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth/preds/preds.json electra --na-prob-file /content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/second/preds/na_probs.json  --na-prob-thresh 0.5  
    else:
        !python /content/Arabic-MRC/evaluatev2.py /content/asquadv2-val.json /content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth/preds/preds.json electra  --out-file /content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth
    if log_path:
        with open(os.path.join(log_path, 'res.csv')) as f:
            DictReader_obj = csv.DictReader(f)
            lastrow = None
            for item in DictReader_obj:
                lastrow = dict(item)
        #print(lastrow)
        return lastrow
    return 1

In [ ]:
#@title
def span_train(model,start_epoch, num_epochs, optimizer,max_compined_metric, train_loader, val_loader, log, exp_name):
  curr_ckp_path, best_ckp_path, exp_path = order_exp('/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span', exp_name)
  model.train()
  for epoch in range(start_epoch,num_epochs):
    total_loss = 0.0
    loop = tqdm(train_loader, leave=True)
    for batch_idx, batch in enumerate(loop):
      tokens = batch['input_ids'].to(device)
      masks = batch['attention_mask'].to(device)
      tokens_type = batch['token_type_ids'].to(device)
      gt_start = batch['start_positions'].to(device)
      gt_end = batch['end_positions'].to(device)
      outputs = model(tokens, masks, tokens_type, start_positions=gt_start, end_positions=gt_end)
      loss = outputs.loss
      loss = 2*loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      total_loss = total_loss + ((1 / (batch_idx + 1)) * (loss.item() - total_loss)) 
      loop.set_description(f'Epoch {epoch}')
      loop.set_postfix(loss=loss.item())
    
    imd_preds, script_preds = get_raw_preds(val_loader, model,val_ids_to_idx,val_offset,asquad_val_contexts, 30, 10)
    result_dict = get_preds(script_preds, None,'/content/asquadv2-val.json',exp_path )
    checkpoint = {
            'epoch': epoch + 1,
            'result_dict':result_dict,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
    curr_compined_metric = float(result_dict['HasAns_exact'])+1.5*float(result_dict['HasAns_f1'])
    if curr_compined_metric>=max_compined_metric:
      max_compined_metric = curr_compined_metric
      save_ckp(checkpoint, True, curr_ckp_path, best_ckp_path)
    else:
      save_ckp(checkpoint, False, curr_ckp_path, best_ckp_path)
    print("ckp saved")
  return model

In [ ]:
#@title
QA_AraElectra = ElectraForQuestionAnswering.from_pretrained(model_namexx)
freeze(QA_AraElectra.electra, 6)
span_num_epochs = 4
span_learning_rate = 3e-5
span_optimizer = torch.optim.AdamW(QA_AraElectra.parameters(), lr=span_learning_rate)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#criterion_span = nn.CrossEntropyLoss(reduction='none')
#cls_criterion = nn.CrossEntropyLoss()
QA_AraElectra.to(device)

134602752 42527232


ElectraForQuestionAnswering(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768

In [ ]:
#@title
span_trained_model = span_train(QA_AraElectra,0, 4, span_optimizer,0.0, span_train_loader, val_loader, True, '/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth')

100%|██████████| 1/1 [00:00<00:00,  5.14it/s]


{
  "exact": 16.666666666666668,
  "f1": 29.583333333333332,
  "total": 6,
  "HasAns_exact": 16.666666666666668,
  "HasAns_f1": 29.583333333333332,
  "HasAns_total": 6
}
ckp saved


100%|██████████| 1/1 [00:00<00:00,  5.23it/s]


{
  "exact": 16.666666666666668,
  "f1": 27.083333333333332,
  "total": 6,
  "HasAns_exact": 16.666666666666668,
  "HasAns_f1": 27.083333333333332,
  "HasAns_total": 6
}
ckp saved


100%|██████████| 1/1 [00:00<00:00,  5.10it/s]


{
  "exact": 16.666666666666668,
  "f1": 27.083333333333332,
  "total": 6,
  "HasAns_exact": 16.666666666666668,
  "HasAns_f1": 27.083333333333332,
  "HasAns_total": 6
}
ckp saved


100%|██████████| 1/1 [00:00<00:00,  5.04it/s]


{
  "exact": 33.333333333333336,
  "f1": 42.30769230769231,
  "total": 6,
  "HasAns_exact": 33.333333333333336,
  "HasAns_f1": 42.30769230769231,
  "HasAns_total": 6
}
ckp saved


In [ ]:
#@title
span_trained_model, span_optim , x, xx = load_ckp('/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth/curr.pt', QA_AraElectra,span_optimizer)

In [ ]:
#@title
max_metric = float(xx['HasAns_exact'])+1.5*float(xx['HasAns_f1'])
print(max_metric)

96.7948717948718


In [ ]:
#@title
span_trained_model = span_train(span_trained_model,2, 4, span_optim,max_metric, span_train_loader, val_loader, True, '/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth')

100%|██████████| 1/1 [00:00<00:00,  4.85it/s]


{
  "exact": 33.333333333333336,
  "f1": 35.0,
  "total": 6,
  "HasAns_exact": 33.333333333333336,
  "HasAns_f1": 35.0,
  "HasAns_total": 6
}
ckp saved


100%|██████████| 1/1 [00:00<00:00,  4.92it/s]


{
  "exact": 33.333333333333336,
  "f1": 35.0,
  "total": 6,
  "HasAns_exact": 33.333333333333336,
  "HasAns_f1": 35.0,
  "HasAns_total": 6
}
ckp saved


In [ ]:
#@title
cls_model = ElectraForSequenceClassification.from_pretrained(model_namex, num_labels = 2)
learning_rate = 3e-5
optimizer = torch.optim.AdamW(cls_model.parameters(), lr=learning_rate)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#criterion_span = nn.CrossEntropyLoss(reduction='none')
#cls_criterion = nn.CrossEntropyLoss()
cls_model.to(device)
cls_model , cls_optim, x, xx = load_ckp('/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/cls/third/best.pt', cls_model,optimizer)

In [ ]:
#@title
cls_model.save_pretrained('AraElectra-ASQuADv2-CLS')
araelectra_tokenizer.save_pretrained('AraElectra-ASQuADv2-CLS')

('AraElectra-ASQuADv2-CLS/tokenizer_config.json',
 'AraElectra-ASQuADv2-CLS/special_tokens_map.json',
 'AraElectra-ASQuADv2-CLS/vocab.txt',
 'AraElectra-ASQuADv2-CLS/added_tokens.json',
 'AraElectra-ASQuADv2-CLS/tokenizer.json')

In [ ]:
#@title
qa_model = ElectraForQuestionAnswering.from_pretrained(model_namexx)
span_learning_rate = 3e-5
span_optimizer = torch.optim.AdamW(qa_model.parameters(), lr=span_learning_rate)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
qa_model.to(device)
qa_model , qa_optim, x, xx = load_ckp('/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth/best.pt', qa_model,span_optimizer)

In [ ]:
#@title
qa_model.save_pretrained('AraElectra-ASQuADv2-QA')
araelectra_tokenizer.save_pretrained('AraElectra-ASQuADv2-QA')

('AraElectra-ASQuADv2-QA/tokenizer_config.json',
 'AraElectra-ASQuADv2-QA/special_tokens_map.json',
 'AraElectra-ASQuADv2-QA/vocab.txt',
 'AraElectra-ASQuADv2-QA/added_tokens.json',
 'AraElectra-ASQuADv2-QA/tokenizer.json')

In [ ]:
#@title
qa_model = ElectraForQuestionAnswering.from_pretrained('AraElectra-ASQuADv2-QA')
cls_model = ElectraForSequenceClassification.from_pretrained('AraElectra-ASQuADv2-CLS',num_labels=2)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
qa_model.to(device)
cls_model.to(device)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [ ]:
#@title
cls_res_dict = cls_eval(cls_model, test_loader,'/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/cls/third',None)

In [ ]:
#@title
cls_res_dict

{'acc': 100.0, 'train_loss': None}

In [ ]:
#@title
cls_preds_dict = dict()
with open('/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/cls/third/preds.txt','r') as f:
    for line in f:
        line = line.strip()
        elements = line.split(',')
        cls_preds_dict[elements[0]] = float(elements[1])
print(len(cls_preds_dict))

6


In [ ]:
#@title
imd_preds, script_preds = get_raw_preds(test_loader, qa_model,test_ids_to_idx,test_offset,asquad_test_contexts, 30, 10)

100%|██████████| 1/1 [00:00<00:00,  5.12it/s]


In [ ]:
#@title
print(imd_preds)

{'3': {'score': 10.855261, 'text': 'العبء الدراسي هو عدد ساعات الدراسة خلال الفصل الدراسي الواحد'}, '4': {'score': -3.7347002, 'text': 'هو'}, '6': {'score': 9.322868, 'text': 'المعدل المطلوب للطالب الغير مقيد هو معدل عام 2 . 50 نقاط من أصل 4 . 00 نقاط'}, '5': {'score': 1.7065122, 'text': 'ويتم قبوله قبولا مشروطا بتحويله لطالب مقيد بعد اجتياز 12 وحدة دراسية بمعدل لا يقل عن 3 . 00 نقاط من أصل 4 . 00 نقاط'}, '1': {'score': 0.89066315, 'text': 'نوعان'}, '2': {'score': 9.026323, 'text': 'شروط التقديم على كلية الدراسات العليا متغيرة حسب البرامج المطروحة في كل سنة وتكون الشروط مذكورة في اعلان التقديم الذي يتم نشره على الموقع الالكتروني الخاص بكلية الدراسات العليا'}}


In [ ]:
#@title
def get_preds2(total_preds, no_probs_preds,data_path):
    preds_path = os.path.join('preds')
    if not os.path.exists(preds_path):
        os.mkdir(preds_path)
    no_probs_path = os.path.join(preds_path, 'na_probs.json')
    text_preds_path = os.path.join(preds_path, 'preds.json')
    jsonString = json.dumps(total_preds)
    jsonFile = open(text_preds_path, "w")
    jsonFile.write(jsonString)
    jsonFile.close()
    if no_probs_preds is not None:
        jsonString = json.dumps(no_probs_preds)
        jsonFile = open(no_probs_path, "w")
        jsonFile.write(jsonString)
        jsonFile.close()
        #!python evaluatev2.py data_path text_preds_path electra --na-prob-file no_probs_path --na-prob-thresh 0.4 --out-file log_path
        #os.system(f"python evaluatev2.py {data_path} {text_preds_path} electra --na-prob-file {no_probs_path} --na-prob-thresh 0.5 --out-file {log_path}")
        !python /content/Arabic-MRC/evaluatev2.py /content/asquadv2-test.json /content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth/preds/preds.json electra --na-prob-file /content/preds/na_probs.json  --na-prob-thresh 0.5  
    return 1

In [ ]:
#@title
qa_result_dict = get_preds2(script_preds, cls_preds_dict,'/content/Arabic-MRC/Runs/AraElectraDecoupledAsquadv2/train/span/fourth/preds' )

{
  "exact": 0.0,
  "f1": 1.9607843137254901,
  "total": 6,
  "HasAns_exact": 0.0,
  "HasAns_f1": 1.9607843137254901,
  "HasAns_total": 6,
  "best_exact": 0.0,
  "best_exact_thresh": 0.0,
  "best_f1": 1.9607843137254901,
  "best_f1_thresh": 0.00025343577726744115
}
